In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from config import api_key
import requests
import json

In [ ]:
# Locate CSV files and convert to dataframes
STATE_DATA_PATH = "Resources/MN_2020_ADI_9 Digit Zip Code_v3.2.csv"
LOCAL_DATA_PATH = "Resources/Local_Repository_Data.csv"
PT_DATA_PATH = "Resources/Patient_Data.csv"

state_df = pd.read_csv(STATE_DATA_PATH)
local_df = pd.read_csv(LOCAL_DATA_PATH)
pt_df = pd.read_csv(PT_DATA_PATH)

# DATA CLEANING
---
state_df cleaning

In [ ]:
state_df.head()

In [ ]:
# Number of zip codes in state_df
print(f"There are {state_df.shape[0]} MN zip codes in state_df")

In [ ]:
# Drop unwanted columns from state_df
clean_state_df = state_df.drop(["GISJOIN", "FIPS", "TYPE"], axis=1)
clean_state_df.head()

In [ ]:
# Find bad data to drop
clean_state_df["ADI_STATERANK"].unique()

In [ ]:
# Drop bad data
clean_state_df = clean_state_df.loc[(clean_state_df["ADI_STATERANK"] != "GQ") &
                                    (state_df["ADI_STATERANK"] != "PH") &
                                    (state_df["ADI_STATERANK"] != "GQ-PH")]
clean_state_df["ADI_STATERANK"].unique()

In [ ]:
# Drop NaN data
clean_state_df = clean_state_df.dropna()

In [ ]:
# Confirm that all bad data is dropped from ADI_STATERANK column
clean_state_df["ADI_STATERANK"].unique()

In [ ]:
# Confirm that all bad data is dropped from ADI_NATRANK column
clean_state_df["ADI_NATRANK"].unique()

In [ ]:
# Reset the index after dropping rows
clean_state_df = clean_state_df.reset_index(drop=True)
clean_state_df.head()

In [ ]:
# Print number of rows that were dropped during data cleaning
print(f"{state_df.shape[0] - clean_state_df.shape[0]} rows were dropped from state_df during data cleaning.")

In [ ]:
# Rename zip code column name to match the pt_data_df
clean_state_df = clean_state_df.rename(columns={"ZIP_4": "Zip Code (9-digit)"})
clean_state_df.head()

---
local_df cleaning

In [ ]:
local_df.head()

In [ ]:
# Print the number of repositories
print(f"RoundtableRx has {local_df.shape[0]} local repositories where patients can get their medications.")

In [ ]:
# Drop unwanted columns
clean_local_df = local_df.drop("Established", axis=1)
clean_local_df.head()

In [ ]:
# Check for bad data
clean_local_df["Zip Code (5-digit)"].unique()

In [ ]:
# Check for bad data
clean_local_df["Zip Code (9-digit)"].unique()

----
pt_df cleaning

In [ ]:
pt_df.head()

In [ ]:
# RoundtableRx serves 133 unique patients
print(f"RoundtableRx serves {pt_df.shape[0]} unique patients.")

In [ ]:
# Drop unwanted columns
clean_pt_df = pt_df.drop("Date Started or Renewed", axis=1)
clean_pt_df.head()

In [ ]:
# Check for bad data
clean_pt_df["Zip Code (5-digit)"].unique()

In [ ]:
# Check for bad data
clean_pt_df["Zip Code (9-digit)"].unique()

In [ ]:
# Drop bad data
clean_pt_df = clean_pt_df.loc[(clean_pt_df["Zip Code (9-digit)"] != "??") & (clean_pt_df["Zip Code (5-digit)"] != "??")]
clean_pt_df.head()

In [ ]:
# Reset the index after dropping rows
clean_pt_df = clean_pt_df.reset_index(drop=True)
clean_pt_df.head()

In [ ]:
# Print number of pts lost during data cleaning
print(f"{pt_df.shape[0] - clean_pt_df.shape[0]} patients were dropped during data cleaning.")

# Data Merging

In [ ]:
# Before merging clean_pt_df and clean_state_df, ensure that common column is of the same dtype
clean_pt_df['Zip Code (9-digit)'] = clean_pt_df['Zip Code (9-digit)'].astype(int)
clean_state_df['Zip Code (9-digit)'] = clean_state_df['Zip Code (9-digit)'].astype(int)

In [ ]:
# Merge clean_pt_df and clean_state_df so that we can get the ADI rank of each patient
pt_adi_df = pd.merge(clean_pt_df,clean_state_df, how="inner", on=["Zip Code (9-digit)", "Zip Code (9-digit)"])
pt_adi_df.head()

In [ ]:
# Print number of pts lost during data merging
print(f"{clean_pt_df.shape[0] - pt_adi_df.shape[0]} patients were dropped during data merging.")

In [ ]:
# Determine total number of patients with ADI ranks we can analyze
total_pts = pt_adi_df["Patient ID"].count()
print(f"ADI ranks are available for {total_pts} of RoundtableRx's patients.")

In [ ]:
# Before merging clean_local_df and clean_state_df, ensure that common column is of the same dtype
clean_local_df['Zip Code (9-digit)'] = clean_local_df['Zip Code (9-digit)'].astype(int)

In [ ]:
# Merge clean_local_df and clean_state_df so that we can get the ADI rank of each local repository
local_adi_df = pd.merge(clean_local_df,clean_state_df, how="left", on=["Zip Code (9-digit)", "Zip Code (9-digit)"])
local_adi_df

1.using geoapify (https://apidocs.geoapify.com/docs/geocoding/forward-geocoding/#geocode-zipcodes) add lat and lon columns as well get a county column to pt_adi_df and local_adi_df. get county column added to clean_state_df.
    - Ayan
    Travis

make the maps

2. get mean, mode for pt_adi_df ADI_STATERANK and ADI_NATRANK
    - Kokoe

make bar charts y = # of pts, x = ADIs core for pt_adi_df. state ranks only
    - Roli

3. pie chart ADI scores
    - Joanna

4. health access / health outcomes vs adi score per county

to create a new branch:     git checkout -b new-branch-name
to push to a branch:        git push origin new-branch-name

10 AM GROUP ON SATURDAY

2:30 GROUP ON SATURDAY


In [ ]:
local_adi_df["LAT"]=""
local_adi_df["LONG"] =""
local_adi_df["County"]= ""
local_adi_df

In [ ]:
for index, row in local_adi_df.iterrows():
    # Loop through zip codes
    zip_code = local_adi_df.loc[index, "Zip Code (5-digit)"]
    # define target url
    target_url = f"https://api.geoapify.com/v1/geocode/search?text={zip_code}&lang=en&limit=10&type=postcode&filter=countrycode:us&apiKey={api_key}"
    #get lat, lon, county
    latitude = local_adi_df.loc[index, "LAT"]
    longitude = local_adi_df.loc[index, "LONG"]
    county = local_adi_df.loc[index, "County"]
    # make API request
    response = requests.get(target_url).json()
    try:
        local_adi_df.loc[index, "LAT"] = response["features"][0]["properties"]["lat"]
        local_adi_df.loc[index, "LONG"] = response["features"][0]["properties"]["lon"]
        local_adi_df.loc[index, "County"] = response["features"][0]["properties"]["county"]
    except:
        local_adi_df.loc[index, "LAT"] = "Not Found"
        local_adi_df.loc[index, "LONG"] = "Not Found"
        local_adi_df.loc[index, "County"] = "Not Found"
        
local_adi_df
        

In [ ]:
# Create a function to add latitude, longitude, and County columns to the state, local, and pt dataframes
def column_creation(dataframe):
    dataframe["LAT"]=""
    dataframe["LONG"] =""
    dataframe["County"]= ""  

In [ ]:
# Run column_creation function on pt_adi_df
column_creation(pt_adi_df)
pt_adi_df.head()

In [ ]:
# Run column_creation function on local_adi_df
column_creation(local_adi_df)
local_adi_df.head()

In [ ]:
# Run column_creation function on clean_state_df
column_creation(clean_state_df)
clean_state_df.head()

In [ ]:
# Create a function to populate empty columns on state, local, and pt dataframes
def location_info(dataframe):
    for index, row in dataframe.iterrows():
        # Loop through zip codes
        zip_code = dataframe.loc[index, "Zip Code (5-digit)"]
        # define target url
        target_url = f"https://api.geoapify.com/v1/geocode/search?text={zip_code}&lang=en&limit=10&type=postcode&filter=countrycode:us&apiKey={api_key}"
        #get lat, lon, county
        latitude = dataframe.loc[index, "LAT"]
        longitude = dataframe.loc[index, "LONG"]
        county = dataframe.loc[index, "County"]
        # make API request
        response = requests.get(target_url).json()
        try:
            dataframe.loc[index, "LAT"] = response["features"][0]["properties"]["lat"]
            dataframe.loc[index, "LONG"] = response["features"][0]["properties"]["lon"]
            dataframe.loc[index, "County"] = response["features"][0]["properties"]["county"]
            print(f"{zip_code} found")
        except:
            dataframe.loc[index, "LAT"] = "Not Found"
            dataframe.loc[index, "LONG"] = "Not Found"
            dataframe.loc[index, "County"] = "Not Found"
            print(f"{zip_code} NOT found")


In [ ]:
# Run location_info function on local_adi_df
location_info(local_adi_df)
local_adi_df

In [ ]:
# Run location_info function on pt_adi_df
location_info(pt_adi_df)
pt_adi_df

In [ ]:
# Run location_info function on clean_state_df
location_info(clean_state_df)
clean_state_df.head()